# 🟣 ETL avec Python

**Badge:** 🟣 Expert | ⏱ 60 min | 🔑 **Concepts clés :** pipeline ETL, extract, transform, load, bonnes pratiques

## Objectifs

- Comprendre l'architecture d'un pipeline ETL
- Implémenter les 3 phases : Extract, Transform, Load
- Valider les données avec Pydantic
- Gérer les erreurs et le logging
- Appliquer les bonnes pratiques (idempotence, configuration)
- Construire un pipeline ETL complet

## Prérequis

- Pandas, Pydantic, requests
- Bases de données (SQLite, DuckDB)
- Formats de données (CSV, Parquet)
- Logging Python

## 1. ETL : Extract, Transform, Load

### Qu'est-ce qu'un ETL ?

Un **pipeline ETL** extrait des données depuis des sources, les transforme, puis les charge dans une destination.

```
┌─────────┐     ┌───────────┐     ┌──────┐
│ Extract │ --> │ Transform │ --> │ Load │
└─────────┘     └───────────┘     └──────┘
    │               │                 │
   API           Validate          Database
   CSV           Clean             Parquet
   DB            Enrich            S3
```

### Extract (Extraction)
- Lire depuis des sources : API, fichiers, bases de données
- Gérer les erreurs de connexion
- Pagination pour les APIs

### Transform (Transformation)
- Nettoyer : valeurs manquantes, doublons, formats
- Valider : types, contraintes (Pydantic)
- Enrichir : jointures, calculs, agrégations

### Load (Chargement)
- Écrire dans la destination : database, data lake, warehouse
- Gestion des conflits (upsert)
- Partitionnement

### ELT vs ETL

**ETL** : Transform AVANT Load (sur le serveur local)  
**ELT** : Load PUIS Transform (dans le data warehouse)  

ELT est devenu populaire avec les data warehouses modernes (BigQuery, Snowflake) car ils ont la puissance de calcul.

## 2. Setup : Configuration et logging

In [ ]:
import pandas as pd
import numpy as np
import requests
import logging
from pathlib import Path
from typing import List, Optional
from datetime import datetime, timedelta
from pydantic import BaseModel, Field, field_validator, ValidationError
from pydantic_settings import BaseSettings
import sqlite3
import json

# Configuration du logging structuré
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)-8s | %(name)s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

logger = logging.getLogger('ETL')
logger.info("Logger ETL initialisé")

In [ ]:
# Configuration avec Pydantic Settings
class ETLConfig(BaseSettings):
    """Configuration du pipeline ETL"""
    
    # Sources
    source_csv_path: str = "etl_data/sales_raw.csv"
    source_api_url: str = "https://api.example.com/products"
    
    # Destinations
    output_parquet_path: str = "etl_data/sales_clean.parquet"
    output_db_path: str = "etl_data/analytics.db"
    
    # Paramètres
    batch_size: int = 1000
    max_retries: int = 3
    
    # Dead letter queue (erreurs)
    error_log_path: str = "etl_data/errors.jsonl"
    
    class Config:
        env_prefix = "ETL_"

config = ETLConfig()
logger.info(f"Configuration chargée : {config.model_dump()}")

## 3. Extract : Extraction depuis sources multiples

In [ ]:
# Créer des données sources
Path('etl_data').mkdir(exist_ok=True)

# Source 1 : CSV avec données brutes (parfois incorrectes)
np.random.seed(42)
df_raw = pd.DataFrame({
    'order_id': range(1, 1001),
    'order_date': pd.date_range('2024-01-01', periods=1000, freq='H'),
    'customer_id': np.random.randint(1, 200, 1000),
    'product_id': np.random.randint(1, 50, 1000),
    'quantity': np.random.randint(-2, 10, 1000),  # Certaines négatives !
    'unit_price': np.random.uniform(-10, 2000, 1000),  # Certaines négatives !
    'status': np.random.choice(['completed', 'pending', 'cancelled', 'invalid', None], 1000)
})

df_raw.to_csv(config.source_csv_path, index=False)
logger.info(f"Données source créées : {config.source_csv_path}")

In [ ]:
def extract_from_csv(file_path: str) -> pd.DataFrame:
    """Extrait les données d'un fichier CSV"""
    logger.info(f"Extract: Lecture CSV {file_path}")
    
    try:
        df = pd.read_csv(file_path, parse_dates=['order_date'])
        logger.info(f"Extract: {len(df)} lignes extraites")
        return df
    except Exception as e:
        logger.error(f"Extract: Erreur lecture CSV - {e}")
        raise

# Extraction
df_extracted = extract_from_csv(config.source_csv_path)
print("Aperçu des données extraites :")
print(df_extracted.head())
print(f"\nTypes : \n{df_extracted.dtypes}")

In [ ]:
def extract_from_api(api_url: str, max_retries: int = 3) -> List[dict]:
    """Extrait les données d'une API avec retry"""
    logger.info(f"Extract: Appel API {api_url}")
    
    # Simulation d'une API (en réalité on utiliserait requests.get)
    # for attempt in range(max_retries):
    #     try:
    #         response = requests.get(api_url, timeout=10)
    #         response.raise_for_status()
    #         data = response.json()
    #         logger.info(f"Extract: {len(data)} enregistrements de l'API")
    #         return data
    #     except requests.RequestException as e:
    #         logger.warning(f"Extract: Tentative {attempt+1}/{max_retries} échouée - {e}")
    #         if attempt == max_retries - 1:
    #             raise
    #         time.sleep(2 ** attempt)  # Exponential backoff
    
    # Données simulées
    products = [
        {'product_id': i, 'product_name': f'Product_{i}', 'category': np.random.choice(['A', 'B', 'C'])}
        for i in range(1, 51)
    ]
    logger.info(f"Extract: {len(products)} produits simulés")
    return products

# Extraction API
products_data = extract_from_api(config.source_api_url)
df_products = pd.DataFrame(products_data)
print("\nProduits extraits de l'API :")
print(df_products.head())

## 4. Transform : Validation avec Pydantic

In [ ]:
# Modèles Pydantic pour validation
class OrderSchema(BaseModel):
    """Schéma de validation pour une commande"""
    order_id: int = Field(gt=0, description="ID de commande positif")
    order_date: datetime
    customer_id: int = Field(gt=0)
    product_id: int = Field(gt=0)
    quantity: int = Field(gt=0, description="Quantité strictement positive")
    unit_price: float = Field(gt=0, description="Prix strictement positif")
    status: str = Field(pattern=r'^(completed|pending|cancelled)$')
    
    @field_validator('order_date')
    @classmethod
    def validate_date(cls, v):
        # Date ne doit pas être dans le futur
        if v > datetime.now():
            raise ValueError("Date dans le futur")
        # Date pas trop ancienne (> 5 ans)
        if v < datetime.now() - timedelta(days=365*5):
            raise ValueError("Date trop ancienne")
        return v

class ProductSchema(BaseModel):
    product_id: int = Field(gt=0)
    product_name: str = Field(min_length=1)
    category: str

logger.info("Schémas de validation définis")

In [ ]:
def transform_validate_orders(df: pd.DataFrame) -> tuple[pd.DataFrame, List[dict]]:
    """Valide et nettoie les commandes"""
    logger.info(f"Transform: Validation de {len(df)} commandes")
    
    valid_records = []
    invalid_records = []
    
    for idx, row in df.iterrows():
        try:
            # Valider avec Pydantic
            validated = OrderSchema(**row.to_dict())
            valid_records.append(validated.model_dump())
        except ValidationError as e:
            # Enregistrer l'erreur
            error = {
                'row_index': int(idx),
                'data': row.to_dict(),
                'errors': e.errors(),
                'timestamp': datetime.now().isoformat()
            }
            invalid_records.append(error)
            logger.warning(f"Transform: Ligne {idx} invalide - {e.error_count()} erreurs")
    
    df_valid = pd.DataFrame(valid_records)
    logger.info(f"Transform: {len(df_valid)} valides, {len(invalid_records)} invalides")
    
    return df_valid, invalid_records

# Transformation et validation
df_valid, errors = transform_validate_orders(df_extracted)

print(f"\n✓ Validation terminée :")
print(f"  Valides : {len(df_valid)}")
print(f"  Invalides : {len(errors)}")

if errors:
    print(f"\nPremière erreur (exemple) :")
    print(json.dumps(errors[0], indent=2, default=str))

## 5. Transform : Enrichissement

In [ ]:
def transform_enrich(df_orders: pd.DataFrame, df_products: pd.DataFrame) -> pd.DataFrame:
    """Enrichit les commandes avec les informations produits"""
    logger.info(f"Transform: Enrichissement de {len(df_orders)} commandes")
    
    # Jointure avec produits
    df_enriched = df_orders.merge(
        df_products,
        on='product_id',
        how='left'
    )
    
    # Calculs
    df_enriched['total_amount'] = (df_enriched['quantity'] * df_enriched['unit_price']).round(2)
    
    # Colonnes dérivées
    df_enriched['year'] = df_enriched['order_date'].dt.year
    df_enriched['month'] = df_enriched['order_date'].dt.month
    df_enriched['day_of_week'] = df_enriched['order_date'].dt.day_name()
    df_enriched['is_weekend'] = df_enriched['order_date'].dt.dayofweek >= 5
    
    # Catégories de montant
    df_enriched['amount_category'] = pd.cut(
        df_enriched['total_amount'],
        bins=[0, 100, 500, 1000, float('inf')],
        labels=['Small', 'Medium', 'Large', 'XLarge']
    )
    
    logger.info(f"Transform: Enrichissement terminé - {len(df_enriched)} lignes")
    return df_enriched

# Enrichissement
df_enriched = transform_enrich(df_valid, df_products)

print("\n✓ Données enrichies :")
print(df_enriched.head())
print(f"\nColonnes : {list(df_enriched.columns)}")

## 6. Load : Chargement vers destinations multiples

In [ ]:
def load_to_parquet(df: pd.DataFrame, output_path: str, partition_cols: Optional[List[str]] = None):
    """Charge les données dans un fichier Parquet"""
    logger.info(f"Load: Écriture Parquet {output_path}")
    
    try:
        if partition_cols:
            df.to_parquet(
                output_path,
                partition_cols=partition_cols,
                index=False,
                compression='snappy'
            )
            logger.info(f"Load: Parquet partitionné par {partition_cols}")
        else:
            df.to_parquet(
                output_path,
                index=False,
                compression='snappy'
            )
        
        file_size = Path(output_path).stat().st_size / 1024**2 if not partition_cols else 0
        logger.info(f"Load: {len(df)} lignes écrites ({file_size:.2f} MB)")
        
    except Exception as e:
        logger.error(f"Load: Erreur écriture Parquet - {e}")
        raise

# Chargement Parquet
load_to_parquet(df_enriched, config.output_parquet_path)
print(f"\n✓ Données chargées : {config.output_parquet_path}")

In [ ]:
def load_to_database(df: pd.DataFrame, db_path: str, table_name: str, if_exists: str = 'replace'):
    """Charge les données dans une base SQLite"""
    logger.info(f"Load: Écriture database {db_path} table {table_name}")
    
    try:
        with sqlite3.connect(db_path) as conn:
            df.to_sql(
                name=table_name,
                con=conn,
                if_exists=if_exists,
                index=False
            )
            
            # Vérifier
            cursor = conn.cursor()
            cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
            count = cursor.fetchone()[0]
            
            logger.info(f"Load: {count} lignes dans {table_name}")
            
    except Exception as e:
        logger.error(f"Load: Erreur écriture database - {e}")
        raise

# Chargement database
load_to_database(df_enriched, config.output_db_path, 'orders_clean')
print(f"✓ Données chargées : {config.output_db_path}")

In [ ]:
def load_errors_to_dlq(errors: List[dict], error_log_path: str):
    """Enregistre les erreurs dans une Dead Letter Queue (JSONL)"""
    logger.info(f"Load: Écriture erreurs {error_log_path}")
    
    try:
        with open(error_log_path, 'a') as f:
            for error in errors:
                f.write(json.dumps(error, default=str) + '\n')
        
        logger.info(f"Load: {len(errors)} erreurs enregistrées")
        
    except Exception as e:
        logger.error(f"Load: Erreur écriture DLQ - {e}")

# Dead Letter Queue
if errors:
    load_errors_to_dlq(errors, config.error_log_path)
    print(f"✓ Erreurs enregistrées : {config.error_log_path}")

## 7. Pipeline ETL complet : Orchestration

In [ ]:
from dataclasses import dataclass
from typing import Dict, Any

@dataclass
class ETLMetrics:
    """Métriques du pipeline ETL"""
    start_time: datetime
    end_time: datetime
    records_extracted: int
    records_valid: int
    records_invalid: int
    records_loaded: int
    
    @property
    def duration_seconds(self) -> float:
        return (self.end_time - self.start_time).total_seconds()
    
    def to_dict(self) -> Dict[str, Any]:
        return {
            'start_time': self.start_time.isoformat(),
            'end_time': self.end_time.isoformat(),
            'duration_seconds': self.duration_seconds,
            'records_extracted': self.records_extracted,
            'records_valid': self.records_valid,
            'records_invalid': self.records_invalid,
            'records_loaded': self.records_loaded,
            'success_rate': round(self.records_valid / self.records_extracted * 100, 2)
        }

class ETLPipeline:
    """Pipeline ETL complet"""
    
    def __init__(self, config: ETLConfig):
        self.config = config
        self.logger = logging.getLogger('ETLPipeline')
    
    def run(self) -> ETLMetrics:
        """Exécute le pipeline ETL complet"""
        start_time = datetime.now()
        self.logger.info("="*70)
        self.logger.info("Démarrage du pipeline ETL")
        self.logger.info("="*70)
        
        try:
            # 1. EXTRACT
            self.logger.info("PHASE 1: EXTRACT")
            df_orders = extract_from_csv(self.config.source_csv_path)
            products_data = extract_from_api(self.config.source_api_url)
            df_products = pd.DataFrame(products_data)
            records_extracted = len(df_orders)
            
            # 2. TRANSFORM
            self.logger.info("PHASE 2: TRANSFORM")
            df_valid, errors = transform_validate_orders(df_orders)
            df_enriched = transform_enrich(df_valid, df_products)
            
            # 3. LOAD
            self.logger.info("PHASE 3: LOAD")
            load_to_parquet(df_enriched, self.config.output_parquet_path)
            load_to_database(df_enriched, self.config.output_db_path, 'orders_clean')
            
            if errors:
                load_errors_to_dlq(errors, self.config.error_log_path)
            
            # Métriques
            end_time = datetime.now()
            metrics = ETLMetrics(
                start_time=start_time,
                end_time=end_time,
                records_extracted=records_extracted,
                records_valid=len(df_valid),
                records_invalid=len(errors),
                records_loaded=len(df_enriched)
            )
            
            self.logger.info("="*70)
            self.logger.info("Pipeline ETL terminé avec succès")
            self.logger.info(f"Durée : {metrics.duration_seconds:.2f}s")
            self.logger.info("="*70)
            
            return metrics
            
        except Exception as e:
            self.logger.error(f"Pipeline ETL échoué : {e}", exc_info=True)
            raise

# Exécuter le pipeline
pipeline = ETLPipeline(config)
metrics = pipeline.run()

print("\n" + "="*70)
print("MÉTRIQUES DU PIPELINE")
print("="*70)
for key, value in metrics.to_dict().items():
    print(f"  {key}: {value}")

## 8. Bonnes pratiques

### Idempotence

Un pipeline **idempotent** produit le même résultat s'il est exécuté plusieurs fois.

In [ ]:
def idempotent_load(df: pd.DataFrame, db_path: str, table_name: str, unique_key: str):
    """Charge avec upsert (idempotent)"""
    logger.info(f"Load idempotent: {table_name} avec clé {unique_key}")
    
    with sqlite3.connect(db_path) as conn:
        # Créer table si inexistante
        df.head(0).to_sql(table_name, conn, if_exists='append', index=False)
        
        # Supprimer les lignes existantes
        existing_ids = tuple(df[unique_key].tolist())
        if existing_ids:
            placeholders = ','.join(['?'] * len(existing_ids))
            conn.execute(f"DELETE FROM {table_name} WHERE {unique_key} IN ({placeholders})", existing_ids)
        
        # Insérer nouvelles lignes
        df.to_sql(table_name, conn, if_exists='append', index=False)
        
        logger.info(f"Load: {len(df)} lignes upserted")

# Test idempotence
idempotent_load(df_enriched, config.output_db_path, 'orders_idempotent', 'order_id')
print("✓ Load idempotent (peut être rejoué sans doublon)")

### Gestion d'erreurs avancée

In [ ]:
from typing import Callable
import time

def retry_with_backoff(func: Callable, max_retries: int = 3, backoff_factor: int = 2):
    """Retry avec exponential backoff"""
    for attempt in range(max_retries):
        try:
            return func()
        except Exception as e:
            if attempt == max_retries - 1:
                logger.error(f"Échec après {max_retries} tentatives")
                raise
            
            wait_time = backoff_factor ** attempt
            logger.warning(f"Tentative {attempt+1}/{max_retries} échouée. Retry dans {wait_time}s")
            time.sleep(wait_time)

# Exemple d'utilisation
def unreliable_operation():
    # Simulation opération qui échoue parfois
    import random
    if random.random() < 0.3:  # 30% échec
        raise Exception("Erreur temporaire")
    return "Success"

# result = retry_with_backoff(unreliable_operation)
print("✓ Retry avec exponential backoff implémenté")

### Tests unitaires (exemple)

In [ ]:
# Exemple de test pour les transformations
def test_transform_enrich():
    """Test de la fonction d'enrichissement"""
    # Données de test
    df_orders_test = pd.DataFrame({
        'order_id': [1, 2],
        'order_date': [datetime(2024, 1, 1), datetime(2024, 1, 2)],
        'customer_id': [1, 2],
        'product_id': [1, 2],
        'quantity': [1, 2],
        'unit_price': [100.0, 200.0],
        'status': ['completed', 'completed']
    })
    
    df_products_test = pd.DataFrame({
        'product_id': [1, 2],
        'product_name': ['Product_1', 'Product_2'],
        'category': ['A', 'B']
    })
    
    # Transformation
    result = transform_enrich(df_orders_test, df_products_test)
    
    # Assertions
    assert len(result) == 2, "Devrait retourner 2 lignes"
    assert 'total_amount' in result.columns, "Colonne total_amount manquante"
    assert result['total_amount'].iloc[0] == 100.0, "Calcul incorrect"
    assert result['total_amount'].iloc[1] == 400.0, "Calcul incorrect"
    assert 'product_name' in result.columns, "Jointure échouée"
    
    print("✓ Test transform_enrich passed")

test_transform_enrich()
print("\n💡 Tests unitaires essentiels pour la fiabilité du pipeline")

## 9. Orchestration : Vers Airflow / Prefect

Pour des pipelines production, utilisez un orchestrateur.

### Apache Airflow
```python
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime

with DAG(
    'etl_pipeline',
    start_date=datetime(2024, 1, 1),
    schedule_interval='@daily',
    catchup=False
) as dag:
    
    extract_task = PythonOperator(
        task_id='extract',
        python_callable=extract_from_csv
    )
    
    transform_task = PythonOperator(
        task_id='transform',
        python_callable=transform_validate_orders
    )
    
    load_task = PythonOperator(
        task_id='load',
        python_callable=load_to_database
    )
    
    extract_task >> transform_task >> load_task
```

### Prefect (moderne)
```python
from prefect import flow, task

@task
def extract():
    return extract_from_csv('data.csv')

@task
def transform(df):
    return transform_validate_orders(df)

@task
def load(df):
    load_to_database(df, 'db.db', 'table')

@flow
def etl_pipeline():
    df = extract()
    df_clean, errors = transform(df)
    load(df_clean)
```

## Pièges courants

### 1. Pas de gestion d'erreurs

In [ ]:
# ❌ Pipeline sans gestion d'erreurs
# df = pd.read_csv('file.csv')  # Crash si fichier absent
# df.to_sql('table', conn)  # Crash si DB inaccessible

# ✅ Toujours gérer les erreurs
try:
    df = pd.read_csv('file.csv')
except FileNotFoundError:
    logger.error("Fichier introuvable")
    # Fallback ou notification

print("✓ Gérez TOUTES les erreurs possibles")

### 2. Pas de logging

In [ ]:
# ❌ Pas de logging = impossible de débugger
# df = transform(df)

# ✅ Logging à chaque étape
logger.info(f"Transform: {len(df)} lignes avant")
# df = transform(df)
logger.info(f"Transform: {len(df)} lignes après")

print("✓ Le logging est votre meilleur ami")

### 3. Couplage fort

In [ ]:
# ❌ Couplage fort : code difficilement testable
# def etl():
#     df = pd.read_csv('hardcoded.csv')
#     df.to_sql('hardcoded_table', hardcoded_conn)

# ✅ Injection de dépendances
def etl(source: str, destination: str, conn):
    df = pd.read_csv(source)
    df.to_sql(destination, conn)

print("✓ Utilisez la configuration et l'injection de dépendances")

## Mini-exercices

### Exercice 1 : Pipeline avec logging

Créez un mini pipeline qui :  
1. Lit un CSV avec 100 lignes  
2. Filtre les lignes où une colonne > seuil  
3. Écrit le résultat en Parquet  
4. Log chaque étape avec le nombre de lignes

In [ ]:
# Votre code ici


### Exercice 2 : Validation avec Pydantic

1. Créez un schéma Pydantic pour un utilisateur (nom, email, age)  
2. Créez un DataFrame avec des données valides et invalides  
3. Validez et séparez valides/invalides  
4. Enregistrez les erreurs en JSONL

In [ ]:
# Votre code ici


### Exercice 3 : Pipeline ETL complet fil rouge

Créez un pipeline ETL complet :  
1. Extract : Générez 1000 ventes e-commerce  
2. Transform : Validez avec Pydantic, enrichissez avec catégories  
3. Load : Sauvez en Parquet partitionné par mois + SQLite  
4. Métriques : Affichez durée, taux de succès

In [ ]:
# Votre code ici


## Solutions des exercices

In [ ]:
# Solution Exercice 1
logger_ex1 = logging.getLogger('Exercise1')

# 1. Créer CSV
df_ex1 = pd.DataFrame({
    'id': range(1, 101),
    'value': np.random.randint(1, 100, 100)
})
df_ex1.to_csv('etl_data/ex1_input.csv', index=False)
logger_ex1.info(f"CSV créé : {len(df_ex1)} lignes")

# 2. Lire
df_read = pd.read_csv('etl_data/ex1_input.csv')
logger_ex1.info(f"CSV lu : {len(df_read)} lignes")

# 3. Filtrer
threshold = 50
df_filtered = df_read[df_read['value'] > threshold]
logger_ex1.info(f"Filtrage value > {threshold} : {len(df_filtered)} lignes restantes")

# 4. Écrire Parquet
df_filtered.to_parquet('etl_data/ex1_output.parquet', index=False)
logger_ex1.info(f"Parquet écrit : {len(df_filtered)} lignes")

print("\n✓ Exercice 1 terminé")

In [ ]:
# Solution Exercice 2
from pydantic import EmailStr

class UserSchema(BaseModel):
    name: str = Field(min_length=1)
    email: EmailStr
    age: int = Field(ge=18, le=120)

# Données mixtes
df_users = pd.DataFrame({
    'name': ['Alice', 'Bob', '', 'Diana', 'Eve'],
    'email': ['alice@test.com', 'invalid-email', 'bob@test.com', 'diana@test.com', 'eve@test.com'],
    'age': [25, 30, 17, 150, 22]
})

valid_users = []
invalid_users = []

for idx, row in df_users.iterrows():
    try:
        validated = UserSchema(**row.to_dict())
        valid_users.append(validated.model_dump())
    except ValidationError as e:
        invalid_users.append({
            'row': int(idx),
            'data': row.to_dict(),
            'errors': [err['msg'] for err in e.errors()]
        })

# Sauver erreurs
with open('etl_data/ex2_errors.jsonl', 'w') as f:
    for error in invalid_users:
        f.write(json.dumps(error) + '\n')

print(f"\n✓ Exercice 2 terminé :")
print(f"  Valides : {len(valid_users)}")
print(f"  Invalides : {len(invalid_users)}")

In [ ]:
# Solution Exercice 3 (pipeline complet)
class SaleSchema(BaseModel):
    sale_id: int = Field(gt=0)
    date: datetime
    product: str
    amount: float = Field(gt=0)

# 1. Extract
df_sales_ex3 = pd.DataFrame({
    'sale_id': range(1, 1001),
    'date': pd.date_range('2024-01-01', periods=1000, freq='6h'),
    'product': np.random.choice(['A', 'B', 'C'], 1000),
    'amount': np.random.uniform(-10, 500, 1000)  # Certaines négatives
})

logger.info(f"Extract: {len(df_sales_ex3)} ventes générées")

# 2. Transform
valid_sales = []
invalid_sales = []

for idx, row in df_sales_ex3.iterrows():
    try:
        validated = SaleSchema(**row.to_dict())
        valid_sales.append(validated.model_dump())
    except ValidationError:
        invalid_sales.append(row.to_dict())

df_valid_ex3 = pd.DataFrame(valid_sales)
df_valid_ex3['month'] = df_valid_ex3['date'].dt.month
df_valid_ex3['category'] = df_valid_ex3['product'].map({'A': 'Cat1', 'B': 'Cat2', 'C': 'Cat3'})

logger.info(f"Transform: {len(df_valid_ex3)} valides, {len(invalid_sales)} invalides")

# 3. Load
df_valid_ex3.to_parquet(
    'etl_data/sales_ex3_partitioned',
    partition_cols=['month'],
    index=False
)
logger.info("Load: Parquet partitionné écrit")

with sqlite3.connect('etl_data/ex3.db') as conn:
    df_valid_ex3.to_sql('sales', conn, if_exists='replace', index=False)
logger.info("Load: SQLite écrit")

# 4. Métriques
success_rate = len(df_valid_ex3) / len(df_sales_ex3) * 100
print(f"\n✓ Exercice 3 terminé :")
print(f"  Extraites : {len(df_sales_ex3)}")
print(f"  Valides : {len(df_valid_ex3)}")
print(f"  Taux succès : {success_rate:.1f}%")

## Résumé

### Points clés

1. **ETL** = Extract (sources) → Transform (validation, enrichissement) → Load (destinations)
2. **Pydantic** : validation de schéma essentielle pour la qualité des données
3. **Logging** : indispensable pour le debugging et le monitoring
4. **Gestion d'erreurs** : Dead Letter Queue pour les données invalides
5. **Idempotence** : pipeline peut être rejoué sans doublon
6. **Configuration** : externalisez avec Pydantic Settings
7. **Métriques** : trackez durée, taux de succès, volumétrie
8. **Orchestration** : Airflow, Prefect, Dagster pour production

### Checklist pipeline production

- [ ] Logging structuré à chaque étape
- [ ] Validation des données (Pydantic)
- [ ] Gestion d'erreurs complète
- [ ] Dead Letter Queue pour données invalides
- [ ] Idempotence (upsert)
- [ ] Configuration externalisée
- [ ] Tests unitaires
- [ ] Métriques (durée, success rate)
- [ ] Monitoring et alertes
- [ ] Documentation

### Prochaines étapes

- Cheatsheet : Récapitulatif de tous les concepts
- Approfondir : Airflow, dbt, Great Expectations